##0. Install Poetry

In [ ]:
#!curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python -

In [ ]:
#!source $HOME/.poetry/env

In [ ]:

#!poetry init

In [ ]:
#%env PATH="/root/.local/bin:$PATH"

In [ ]:
#for datasets
!pip install pandas
import pandas as pd
!pip install datasets
from datasets import load_dataset

#for embeddings
from transformers import RobertaTokenizer, RobertaModel
from transformers import BertTokenizer, BertModel
import torch

#for pinecone
!pip install --upgrade pinecone-client
import os
import pinecone
from pinecone import Pinecone, ServerlessSpec

#for ngrams
!pip install nltk
import nltk
nltk.download('punkt')
from nltk import ngrams

#for data cleaning
import re
import ast
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

#for NER
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


##1. Preprocessing

###1.1 Preprocessing functions

In [ ]:
#Cleaning text
def clean_text(text):
    # Handle missing values
    if pd.isna(text):
        return text

    # Remove all punctuation except letters, numbers, and spaces
    cleaned = re.sub(r"[^a-zA-Z0-9\s]", "", text.lower())

    # cleaned = " ".join([word for word in cleaned.split(" ") if word not in stopwords])

    # # Extract names using NER and include them in the cleaned text
    # names = extract_names(text)
    # cleaned += " " + " ".join(names)

    # Remove extra whitespace
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

# #Creating n-grams
# def create_ngrams(text, n=2):
#     unigrams = text.split()
#     for i in range(1, n):
#       ng = [" ".join(i) for i in list(ngrams(text.split(), 3))]
#       unigrams.extend(ng)
#     unigrams.append(text)
#     return set(unigrams)

from copy import deepcopy

In [ ]:
# tokenizer = RobertaTokenizer.from_pretrained('distilbert/distilroberta-base') #('smallbenchnlp/roberta-small')
# model = RobertaModel.from_pretrained('distilbert/distilroberta-base') #('smallbenchnlp/roberta-small')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Function to generate RoBERTa embeddings for a list of text items
def generate_embeddings(text):
    # Tokenize and convert to tensors
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Generate RoBERTa embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embeddings as the overall representation
    # Token used to generate a fixed-size representation of the entire input sequence
    # cls_embeddings = outputs.last_hidden_state[0, 0, :] #CLS Embeddings won't work much better
    cls_embeddings = outputs.last_hidden_state.mean(dim=1)[0] # Try pooling all the word vectors using mean
    cls_embeddings = cls_embeddings.tolist()
    return cls_embeddings

###1.2 Preprocessing the dataset

In [ ]:
dataset = load_dataset("AiresPucrs/tmdb-5000-movies") #"Coder-Dragon/wikipedia-movies")
train_dataset = dataset['train']

Generating train split:   0%|          | 0/4803 [00:00<?, ? examples/s]

In [ ]:
df = train_dataset.to_pandas().iloc[:500]
df = df.loc[:, ['id', 'title', 'overview', 'genres', 'keywords', 'homepage']]
df = df.rename(columns={"id": "index"})

def extract_values(text:str) -> list:
  values = ast.literal_eval(text)
  return [i["name"] for i in values]

# Function to extract person names using spaCy NER
def extract_names(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

df["genres"] = df["genres"].apply(lambda x: extract_values(x))
df["keywords"] = df["keywords"].apply(lambda x: extract_values(x))

# df["overview_ngrams"] = df["overview"].apply(lambda x: clean_text(x))
# df["overview_ngrams"] = df["overview_ngrams"].apply(lambda x: create_ngrams(x))

for i in df.index:
  # df.at[i, "vectors"] = df["title"][i]+" "+ df["overview"][i] + " "+" ".join(df["keywords"][i])
  df.at[i, "vectors"] = df["overview"][i]

df["vectors"] = df["vectors"].apply(lambda x: clean_text(x))
#df["vectors"] = df["vectors"].apply(lambda x: create_ngrams(x))
df["vectors"] = df["vectors"].apply(lambda x: generate_embeddings(x))

df.head()

,index,title,overview,genres,keywords,homepage,vectors
0,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[Crime, Comedy]","[hotel, new year's eve, witch, bet, hotel room...",None,"[0.006611584220081568, 0.33808189630508423, 0...."
1,11,Star Wars,Princess Leia is captured and held hostage by ...,"[Adventure, Action, Science Fiction]","[android, galaxy, hermit, death star, lightsab...",http://www.starwars.com/films/star-wars-episod...,"[-0.3931269943714142, 0.04327482357621193, 0.0..."
2,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[Animation, Family]","[father son relationship, harbor, underwater, ...",http://movies.disney.com/finding-nemo,"[-0.22536508738994598, 0.23053354024887085, 0...."
3,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[Comedy, Drama, Romance]","[vietnam veteran, hippie, mentally disabled, r...",None,"[-0.2663663923740387, 0.29598280787467957, -0...."
4,14,American Beauty,"Lester Burnham, a depressed suburban father in...",[Drama],"[male nudity, female nudity, adultery, midlife...",http://www.dreamworks.com/ab/,"[-0.24975287914276123, -0.05232516676187515, 0..."


##2. Inserting Data into Pinecone

In [ ]:
# Set your API key as an environment variable
os.environ['PINECONE_API_KEY'] = '93e962ec-33f4-4697-baeb-d0af041b9d96'

# Create a Pinecone client instance with the API key
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

# Use lower case alphanumeric or hyphens for index name
index_name = 'our-cool-project'

# Clean out the previous database
if index_name in pc.list_indexes().names(): pc.delete_index(index_name)

# Create the index if it doesn't exist
pc.create_index(
    name=index_name,
    dimension=768,  # Adjust based on your embeddings
    metric='euclidean',  # Choose your preferred metric
    spec=ServerlessSpec(cloud='aws', region='us-east-1')  # Match to our setup
  )

# Connect to the existing or newly created index with the correct host
pinecone_df = pc.Index(index_name)

# Confirm the connection
print("Connected to index:", index_name)

Connected to index: our-cool-project


In [ ]:
# Prepare the data for insertion into Pinecone
vectors_to_insert = []

# Iterate over the DataFrame to prepare the data
for idx, row in df.iterrows():
  embeddings = row["vectors"]

  metadata = {}
  for column in row.items():
    if column[0] not in ["vectors"]:
      metadata[column[0]] = str(column[1])

  vectors_to_insert.append({"id": str(idx), "values": embeddings, "metadata": metadata})

# Insert data into Pinecone
pinecone_df.upsert(vectors_to_insert)
print("Data has been inserted into Pinecone.")

Data has been inserted into Pinecone.


### 2.1 Vector Search (Awais - Optional)

In [ ]:
golden_queries = {
     "fighting with lightsabers agains the space empire": "Star Wars",
     "animation a fish father searching his son in the ocean": "Finding Nemo",
     "pirate captain saves a govenrnor's daugther": "Pirates of the Caribbean: The Curse of the Black Pearl",
     "a driver in the future saves a girl and also humanity": "The Fifth Element",
     "science fiction humans and a super computer fight on a space ship": "2001: A Space Odyssey",
     "adventure teenager from the eighties goes back to the past with an eccentric inventor": "Back to the Future",
     "drama two gay cowboys face homophobia": "Brokeback Mountain", "comedy a guy is stuck in a time loop": "Groundhog Day",
     "horror a dead child molester haunts dreams of  teenagers": "A Nightmare on Elm Street",
     "zombie virus turns people into maniacs": "28 Days Later",
     "saddistic killer plays survival games with his victims": "Saw",
     "family animated disney film about a girl and seven dwarves": "Snow White and the Seven Dwarfs",
     "an insomniac and a soap maker hate people and like to fight": "Fight Club",
     "a fantasy adventure girl with scarecrow search wisard": "The Wizard of Oz",
     "a child sees ghosts needs a psychologist": "The Sixth Sense",
     "husband and wife are secrectly rival assasins": "Mr. & Mrs. Smith",
     "genius guy from paris creates perfume from girls": "Perfume: The Story of a Murderer",
     "stalker doctor hires a detective to find his old date": "There's Something About Mary",
     "drama people struggle with drig addiction": "Requiem for a Dream",
     "musical eccentric chocolate maker": "Willy Wonka & the Chocolate Factory"
     }

In [ ]:
query = "animation a fish father searching his son in the ocean"
query = clean_text(query)
query_embeddings = generate_embeddings(query)

results = pinecone_df.query(
      vector=query_embeddings,
      top_k=10,
      include_metadata=True
      )

print(results)

{'matches': [{'id': '2',
              'metadata': {'genres': "['Animation', 'Family']",
                           'homepage': 'http://movies.disney.com/finding-nemo',
                           'index': '12',
                           'keywords': "['father son relationship', 'harbor', "
                                       "'underwater', 'fish tank', 'great "
                                       "barrier reef', 'missing child', "
                                       "'aftercreditsstinger', "
                                       "'duringcreditsstinger', 'short term "
                                       "memory loss', 'clownfish', 'father son "
                                       "reunion', 'protective father']",
                           'overview': 'Nemo, an adventurous young clownfish, '
                                       'is unexpectedly taken from his Great '
                                       "Barrier Reef home to a dentist's "
                            

##3. Querying the Data

###3.1 Filtering the Data

In [ ]:
movie_pool = {}
for movie in df["index"]:
  movie_pool[movie] = 0

In [ ]:
genres = []
for genre in df["genres"]:
  genres.extend([g.lower() for g in genre])
genres = set(genres)

In [ ]:
def filter_data(query):
    query_cleaned = clean_text(query)
    query_ngrams = create_ngrams(query_cleaned)
    query_movie_pool = deepcopy(movie_pool)

    for key, value in query_movie_pool.items():
        movie = df.query(f"index=={key}").iloc[0]
        keywords = [clean_text(k) for k in movie["keywords"]]
        overview = movie["overview_ngrams"]

        for n in query_ngrams:
            if n in keywords:
                # Assign higher weight to matches in keywords
                query_movie_pool[key] += 5
            elif n in genres:
                # Assign medium weight to matches in genres
                query_movie_pool[key] += 3
            elif n in overview:
                # Assign lower weight to matches in overview
                query_movie_pool[key] += 1

    usable_indices = sorted(query_movie_pool.items(), key=lambda x:x[1], reverse=True)
    usable_indices = [i[0] for i in usable_indices if i[1]!=0]
    return usable_indices

###3.2 Vector Search

In [ ]:
def vector_search(query, indices, k=3):
  query = clean_text(query)
  query_embeddings = generate_embeddings(query)
  results = pinecone_df.query(
      vector=query_embeddings,
      top_k=k,
      filter = {"index": {"$in": indices}},
      include_metadata=True
      )
  if [int(i.metadata["index"]) for i in results.matches] == []:
    results = pinecone_df.query(
      vector=query_embeddings,
      top_k=k,
      include_metadata=True
      )
  return [int(i.metadata["index"]) for i in results.matches]

###3.3 Querying the Data

In [ ]:
#query = "fish is looking for his son nemo"
#query = "a comedy about space"

def querying_data(query, k=3):
  indices = filter_data(query)
  vectors = vector_search(query, indices, k)

  def show_results(results, k):
    if len(results)<k: k = len(results)
    results = df.query(f"index in {results[:k]}")
    for i in range(k):
      print("   "+results["title"].iloc[i]+":")
      print("   "+results["overview"].iloc[i]+"\n")

  def return_titles(results, k):
    if len(results)<k: k = len(results)
    results = df.query(f"index in {results[:k]}")
    titles = []
    for i in range(k):
      titles.append(results["title"].iloc[i])
    return titles

  return {"filter": return_titles(indices, k),
          "vector": return_titles(vectors, k)}

  #print("\n"+"Top results by filtering:")
  #show_results(indices, k)
  #print("Top results by vector search:")
  #show_results(vectors, k)



##4. Evaluation

In [ ]:
golden_queries = {
     "fighting with lightsabers agains the space empire": "Star Wars",
     "animation a fish father searching his son in the ocean": "Finding Nemo",
     "pirate captain saves a govenrnor's daugther": "Pirates of the Caribbean: The Curse of the Black Pearl",
     "a driver in the future saves a girl and also humanity": "The Fifth Element",
     "science fiction humans and a super computer fight on a space ship": "2001: A Space Odyssey",
     "andventure teenager from the eighties goes back to the past with an eccentric inventor": "Back to the Future",
     "drama two gay cowboys face homophobia": "Brokeback Mountain", "comedy a guy is stuck in a time loop": "Groundhog Day",
     "horror a dead child molester haunts dreams of  teenagers": "A Nightmare on Elm Street",
     "zombie virus turns people into maniacs": "28 Days Later",
     "saddistic killer plays survival games with his victims": "Saw",
     "family animated disney film about a girl and seven dwarves": "Snow White and the Seven Dwarfs",
     "an insomniac and a soap maker hate people and like to fight": "Fight Club",
     "a fantasy adventure girl with scarecrow search wisard": "The Wizard of Oz",
     "a child sees ghosts needs a psychologist": "The Sixth Sense",
     "husband and wife are secrectly rival assasins": "Mr. & Mrs. Smith",
     "genius guy from paris creates perfume from girls": "Perfume: The Story of a Murderer",
     "stalker doctor hires a detective to find his old date": "There's Something About Mary",
     "drama people struggle with drig addiction": "Requiem for a Dream",
     "musical eccentric chocolate maker": "Willy Wonka & the Chocolate Factory"
     }

In [ ]:
filter_score = 0
filter_positions = []

vector_score = 0
vector_positions = []

for query in golden_queries.items():
  results = (querying_data(query[0]))
  golden_title = query[1]
  if golden_title in results["filter"]:
    filter_score +=1
    filter_positions.append(results["filter"].index(golden_title))

  if golden_title in results["vector"]:
    vector_score +=1
    vector_positions.append(results["vector"].index(golden_title))

In [ ]:
print (filter_score, filter_positions, vector_score, vector_positions)

13 [0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 1] 2 [0, 2]


##Things we tried:

##Assign different weights to sections:

In [ ]:
def filter_data(query):
    query_cleaned = clean_text(query)
    query_ngrams = create_ngrams(query_cleaned)
    query_movie_pool = deepcopy(movie_pool)

    # Identify keywords in the query
    query_keywords = set()
    for key, value in query_movie_pool.items():
        movie = df.query(f"index=={key}").iloc[0]
        keywords = [clean_text(k) for k in movie["keywords"]]
        query_keywords.update(set(keywords))

    for key, value in query_movie_pool.items():
        movie = df.query(f"index=={key}").iloc[0]
        keywords = [clean_text(k) for k in movie["keywords"]]
        overview = movie["overview_ngrams"]

        for n in query_ngrams:
            if n in keywords:
                # Prioritize matches in keywords
                query_movie_pool[key] += 5
            elif n in genres:
                query_movie_pool[key] += 2
            elif n in overview:
                query_movie_pool[key] += 1

    usable_indices = sorted(query_movie_pool.items(), key=lambda x:x[1], reverse=True)
    usable_indices = [i[0] for i in usable_indices if i[1]!=0]
    return usable_indices


##Mixing indexing: tfidf

1. Compute and Store Document Vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['overview'])

# Store TF-IDF vectors in a dictionary
tfidf_vectors = {index: tfidf_matrix[i].toarray().flatten() for i, index in enumerate(df['index'])}


2. Modify the filter_data function to include TF-IDF vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def filter_data(query):
    query_cleaned = clean_text(query)
    query_ngrams = create_ngrams(query_cleaned)
    query_movie_pool = deepcopy(movie_pool)

    # Compute the TF-IDF vector for the query
    query_vector = vectorizer.transform([query_cleaned]).toarray().flatten()

    for key, value in query_movie_pool.items():
        movie = df.query(f"index=={key}").iloc[0]
        keywords = [clean_text(k) for k in movie["keywords"]]
        overview = movie["overview_ngrams"]

        # Compute the similarity score with the query vector
        doc_vector = tfidf_vectors[key]
        tfidf_score = cosine_similarity([query_vector], [doc_vector])[0][0]

        # Integrate the similarity score into the query_movie_pool
        for n in query_ngrams:
            if n in keywords:
                query_movie_pool[key] += 5
            elif n in genres:
                query_movie_pool[key] += 3
            elif n in overview:
                query_movie_pool[key] += 1

        # Add the TF-IDF similarity score to the pool
        query_movie_pool[key] += tfidf_score * 10  # Adjust the weight as needed

    usable_indices = sorted(query_movie_pool.items(), key=lambda x: x[1], reverse=True)
    usable_indices = [i[0] for i in usable_indices if i[1] != 0]
    return usable_indices


3. Combine Filtering and Vector Search

In [ ]:
def vector_search(query, indices, k=3):
    query_cleaned = clean_text(query)
    query_embeddings = generate_embeddings(query_cleaned)

    if indices:
        results = pinecone_df.query(
            vector=query_embeddings,
            top_k=k,
            filter={"index": {"$in": indices}},
            include_metadata=True
        )
        matches = [int(i.metadata["index"]) for i in results.matches if i.metadata and "index" in i.metadata]
    else:
        matches = []

    if not matches:
        results = pinecone_df.query(
            vector=query_embeddings,
            top_k=k,
            include_metadata=True
        )
        matches = [int(i.metadata["index"]) for i in results.matches if i.metadata and "index" in i.metadata]

    return matches

4. Update querying_data Function

In [ ]:
def querying_data(query=input("Your query: "), k=3):
    indices = filter_data(query)
    vectors = vector_search(query, indices, k)

    def show_results(results, k):
        if len(results) < k:
            k = len(results)
        results_df = df[df['index'].isin(results[:k])]
        for i in range(k):
            print("   " + results_df["title"].iloc[i] + ":")
            print("   " + results_df["overview"].iloc[i] + "\n")

    print("\nTop results by filtering:")
    show_results(indices, k)
    print("Top results by vector search:")
    show_results(vectors, k)

querying_data()

##Mixing indexing: BM25

In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

In [ ]:
# Tokenize documents for BM25
tokenized_docs = [clean_text(doc).split() for doc in df['overview']]
bm25 = BM25Okapi(tokenized_docs)
print("Tokenized Docs:", tokenized_docs[:10])
print("BM25 Model:", bm25)

In [ ]:
# Modified filter_data function
def filter_data(query):
    query_cleaned = clean_text(query)
    query_ngrams = create_ngrams(query_cleaned)
    query_movie_pool = deepcopy({movie: 0 for movie in df["index"]})

    tokenized_query = query_cleaned.split()

    # Debug: Print tokenized query
    print("Tokenized Query:", tokenized_query)

    bm25_scores = bm25.get_scores(tokenized_query)

    # Debug: Print the BM25 scores
    print("BM25 Scores:", bm25_scores[:10])

    # Mapping from index to BM25 scores
    index_to_bm25_score = {index: bm25_scores[i] for i, index in enumerate(df["index"])}

    for key, value in query_movie_pool.items():
        movie = df.query(f"index=={key}").iloc[0]
        keywords = [clean_text(k) for k in movie["keywords"]]
        overview = movie["overview_ngrams"]

        # Get the BM25 score using the mapping
        bm25_score = index_to_bm25_score[key]

        for n in query_ngrams:
            if n in keywords:
                query_movie_pool[key] += 5
            elif n in genres:
                query_movie_pool[key] += 3
            elif n in overview:
                query_movie_pool[key] += 1

        query_movie_pool[key] += bm25_score

    usable_indices = sorted(query_movie_pool.items(), key=lambda x: x[1], reverse=True)
    usable_indices = [i[0] for i in usable_indices if i[1] != 0]
    return usable_indices



In [ ]:
# Modified vector_search function
def vector_search(query, indices, k=3):
    query_cleaned = clean_text(query)
    query_embeddings = generate_embeddings(query_cleaned)[0]  # Correct the dimension

    if indices:
        results = pinecone_df.query(
            vector=query_embeddings,
            top_k=k,
            filter={"index": {"$in": indices}},
            include_metadata=True
        )
        matches = [int(i['metadata']['index']) for i in results['matches'] if i['metadata'] and 'index' in i['metadata']]
    else:
        matches = []

    if not matches:
        results = pinecone_df.query(
            vector=query_embeddings,
            top_k=k,
            include_metadata=True
        )
        matches = [int(i['metadata']['index']) for i in results['matches'] if i['metadata'] and 'index' in i['metadata']]

    return matches

In [ ]:
# querying_data function
def querying_data(query=input("Your query: "), k=3):
    indices = filter_data(query)
    vectors = vector_search(query, indices, k)

    def show_results(results, k):
        if len(results) < k:
            k = len(results)
        results_df = df[df['index'].isin(results[:k])]
        for i in range(k):
            print("   " + results_df["title"].iloc[i] + ":")
            print("   " + results_df["overview"].iloc[i] + "\n")

    print("\nTop results by filtering:")
    show_results(indices, k)
    print("Top results by vector search:")
    show_results(vectors, k)

querying_data()
